In [240]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def filter_location(dat: pd.DataFrame,x:list, y:list):
    '''
    Filtra las coordenadas para Antofagasta, excluyendo las otras comunas
    '''
    try: # Alerts
        dat = dat[(dat['location'].apply(lambda loc: loc['x'] >= x[0] and loc['x'] <= x[1] ))]
        dat = dat[(dat['location'].apply(lambda loc: loc['y'] >= y[0] and loc['y'] <= y[1] ))]
    except KeyError: # Jam
        dat = dat[(dat['line'].apply(lambda line: line[0]['x'] >= x[0] and line[0]['x'] <= x[1]))]
        dat = dat[(dat['line'].apply(lambda line: line[0]['y'] >= y[0] and line[0]['y'] <= y[1]))]

    return dat

URL_DATA = "antofTraffic/data.json"

with open(URL_DATA, "r") as f:
    data = json.load(f)
    f.close()

alerts = pd.DataFrame(data['alerts'])
jams = pd.DataFrame(data['jams'])

PERIM_X = [-70.45534224747098, -70.32743722434367]
PERIM_Y = [-23.701724880116387, -23.411242421131792]

alerts = filter_location(alerts, PERIM_X, PERIM_Y)
jams = filter_location(jams, PERIM_X, PERIM_Y)

In [241]:
alerts.head()

,country,city,reportRating,reportByMunicipalityUser,confidence,reliability,type,uuid,roadType,magvar,subtype,street,location,pubMillis,endreport
0,CI,Antofagasta,1,false,5,10,HAZARD,a123f22e-e5e0-4c6c-8a4e-7434c4fd2110,6,17,HAZARD_ON_ROAD_CONSTRUCTION,Av. República de Croacia,"{'x': -70.412429, 'y': -23.676846}",1724417637000,1.724425e+12
1,CI,Antofagasta,0,false,0,5,JAM,1d8df954-e380-4327-8015-f4fbadf74f2d,6,4,JAM_HEAVY_TRAFFIC,Av. Edmundo Pérez Zujovic,"{'x': -70.392283, 'y': -23.617475}",1724423878000,1.724425e+12
2,CI,Antofagasta,0,false,0,5,JAM,c53a5bc6-f6b6-44b8-a5bd-af69b60a4580,1,359,,Pasaje El Volcán,"{'x': -70.385619, 'y': -23.573323}",1724425406000,1.724425e+12
3,CI,Antofagasta,1,false,0,5,JAM,53b6ff5f-d078-421d-9f78-03e534d78989,2,1,,Azapa,"{'x': -70.384986, 'y': -23.616328}",1724425622000,1.724425e+12
4,CI,Antofagasta,2,false,0,5,ACCIDENT,e87552f7-4572-4096-9fdd-9400ad90e7f8,2,281,,Irarrázaval,"{'x': -70.385121, 'y': -23.603942}",1724425939000,1.724425e+12


In [242]:
alerts['type'].unique()

array(['HAZARD', 'JAM', 'ACCIDENT', 'ROAD_CLOSED'], dtype=object)

In [243]:
naccident = alerts.groupby(['street', 'type', 'subtype']).count()[['country']].rename(columns={'country': 'cantidad'}).loc[:, 'ACCIDENT', :].copy().reset_index()
njam = alerts.groupby(['street', 'type', 'subtype']).count()[['country']].rename(columns={'country': 'cantidad'}).loc[:, 'JAM', :].copy().reset_index()
nhazard = alerts.groupby(['street', 'type', 'subtype']).count()[['country']].rename(columns={'country': 'cantidad'}).loc[:, 'HAZARD', :].copy().reset_index()
nroad_closed = alerts.groupby(['street', 'type', 'subtype']).count()[['country']].rename(columns={'country': 'cantidad'}).loc[:, 'ROAD_CLOSED', :].copy().reset_index()

In [244]:
naccident['prop'] = naccident['cantidad'] / naccident['cantidad'].sum()
njam['prop'] = njam['cantidad'] / njam['cantidad'].sum()
nhazard['prop'] = nhazard['cantidad'] / nhazard['cantidad'].sum()
nroad_closed['prop'] = nroad_closed['cantidad'] / nroad_closed['cantidad'].sum()
naccident

,street,subtype,cantidad,prop
0,Av. Angamos,,1,0.028571
1,Av. Antonio Rendic,,2,0.057143
2,Av. Edmundo Pérez Zujovic,,5,0.142857
3,Av. Edmundo Pérez Zujovic,ACCIDENT_MAJOR,1,0.028571
4,Av. Edmundo Pérez Zujovic,ACCIDENT_MINOR,1,0.028571
5,Av. Grecia,,5,0.142857
6,Av. Grecia,ACCIDENT_MAJOR,1,0.028571
7,Av. Séptimo de Línea,,1,0.028571
8,Av. Óscar Bonilla,,2,0.057143
9,Benito Ocampo,,1,0.028571
